<h1> Case Study: Netflix

<h2> Zaimportuj Pakiety

In [1]:
import pandas as pd
import numpy as np
import warnings
import lxml
import requests
from bs4 import BeautifulSoup
import openpyxl as xl
warnings.filterwarnings("ignore")

<h2> Zaczytaj Dane

In [2]:
content_df = pd.read_csv(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_Productions_Data.csv")
app_reviews_df = pd.read_csv(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_App_Reviews_Data.csv")
users_reviews_df = pd.read_csv(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_Users_Reviews.csv")
critics_reviews_df = pd.read_csv(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_Critics_Reviews.csv")
shows_links_df = pd.read_csv(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_TV_Shows_Links.csv")

<h2> Czyszczenie Danych

<h3> Braki Danych

In [3]:
def count_of_nans():
    nan_counts = content_df.isna().sum()
    nan_counts = nan_counts.to_frame()
    nan_counts = nan_counts.rename(columns={0: '# of NaNs'})
    nan_counts = nan_counts[nan_counts['# of NaNs']!=0]
    nan_counts.sort_values(by='# of NaNs', ascending=False, inplace=True)
    print(nan_counts)

count_of_nans()

                   # of NaNs
seasons                 3744
age_certification       2619
imdb_votes               498
imdb_score               482
imdb_id                  403
tmdb_score               311
tmdb_popularity           91
description               18
title                      1


In [4]:
content_df['seasons'] = np.where(((content_df['type']=='MOVIE')&(content_df['seasons'].isna())), 0, content_df['seasons'])
content_df['seasons'] = np.where(((content_df['type']=='SHOW')&(content_df['seasons'].isna())), -1, content_df['seasons'])

for column in ['imdb_votes', 'imdb_score', 'tmdb_score', 'tmdb_popularity']:
    content_df[column] = content_df[column].fillna(-1)

content_df['description'] = content_df['description'].fillna('unavailable')

content_df['imdb_id'] = content_df['imdb_id'].fillna('unknown')

content_df['age_certification'] = content_df['age_certification'].fillna('not specified')

content_df = content_df.dropna(subset=['title'])

for column in ['genres', 'production_countries']:
    content_df[column] = np.where((content_df[column]=='[]'), 'unknown', content_df[column])

count_of_nans()

Empty DataFrame
Columns: [# of NaNs]
Index: []


<h3> Usuwanie Duplikatów

In [5]:
for column in ['id', 'title', 'imdb_id']: 
    content_df = content_df.drop_duplicates(column)

<h3> Kolumny Warunkowe

In [6]:
content_df['title_small_letters'] = content_df["title"].str.lower()

children_categories = ['G', 'TV-G', 'TV-Y', 'TV-Y7']
supervised_categories = ['PG', 'PG-13', 'TV-14', 'TV-PG']
adults_categories = ['NC-17', 'R', 'TV-MA']

conditions = [content_df['age_certification'].isin(children_categories),    
                content_df['age_certification'].isin(supervised_categories),  
                content_df['age_certification'].isin(adults_categories),
                content_df['age_certification']=='not specified']
                
values = ['children', 'supervised', 'adults', 'not specified']

content_df['age_group'] = np.select(conditions, values)

conditions = [((content_df['type']=='SHOW')&(content_df['runtime']<=30)), 
            ((content_df['type']=='SHOW')&(content_df['runtime']>30)),
            ((content_df['type']=='MOVIE')&(content_df['runtime']<60)),
            ((content_df['type']=='MOVIE')&(content_df['runtime']>=60)&(content_df['runtime']<90)),
            ((content_df['type']=='MOVIE')&(content_df['runtime']>=90))]

values = ['short', 'long', 'short', 'medium', 'long']

content_df['production_runtime_category'] = np.select(conditions, values)

content_df['genres'] = content_df['genres'].str.replace('[', '').str.replace(']', '').str.replace("'", '')

content_df['main_genre'] = content_df['genres'].str.split(',').str.get(0)

content_df['production_countries'] = content_df['production_countries'].str.replace('[', '').str.replace(']', '').str.replace("'", '')

content_df['main_production_country'] = content_df['production_countries'].str.split(',').str.get(0)

<h3> Pobierz Dane z Wikipedii

In [7]:
def scrape_wikipedia_table(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    df = pd.read_html(str(table))[0]
    return df

countries_df = scrape_wikipedia_table('https://en.wikipedia.org/wiki/ISO_3166-2')

countries_df = countries_df.loc[:, ["Entry  (click to view codes)", "Country name (using title case)"]]

countries_df = countries_df.rename(columns={"Entry  (click to view codes)": "Code",
                                             "Country name (using title case)": "Country_Name"})

content_df = content_df.merge(countries_df, left_on='main_production_country', right_on='Code', how='left')

content_df['Country_Name'] = content_df['Country_Name'].fillna('unknown')

<h3> Typy Danych

In [8]:
string_fields = ["id", "title", "type", "description", "age_certification", "genres", "production_countries", "imdb_id", "title_small_letters", "age_group", "production_runtime_category", "main_genre", "main_production_country", "Code", "Country_Name"]

integer_fields = ["release_year", "runtime", "seasons", "imdb_votes"]

float_fields = ["imdb_score", "tmdb_popularity", "tmdb_score"]

for column in string_fields:
    content_df[column] = content_df[column].astype(str)

for column in integer_fields:
    content_df[column] = content_df[column].astype(int)

for column in float_fields:
    content_df[column] = content_df[column].astype(float)

content_df = content_df.loc[:, ['id', 'title', 'title_small_letters', 'type', 'description', 'release_year', 'main_production_country', 'Country_Name', 'main_genre', 'seasons', 'age_certification', 'age_group', 'runtime', 'production_runtime_category', 'imdb_score', 'imdb_votes', 'tmdb_popularity','tmdb_score']]

content_df = content_df.rename(columns={'title_small_letters': 'title_lower', 'main_production_country': 'country_code', 'Country_Name': 'country', 'main_genre': 'genre', 'production_runtime_category': 'runtime_type'})

content_df.reset_index(drop=True, inplace=True)

content_df.head(2)

,id,title,title_lower,type,description,release_year,country_code,country,genre,seasons,age_certification,age_group,runtime,runtime_type,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,five came back: the reference films,SHOW,This collection includes 12 World War II-era p...,1945,US,United States of America,documentation,1,TV-MA,adults,51,long,-1.0,-1,0.600,-1.000
1,tm84618,Taxi Driver,taxi driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,US,United States of America,drama,0,R,adults,114,long,8.2,808582,40.965,8.179


<h3> Przygotowanie Zbioru Danych z Ocenami Krytyków i Widzów 

In [9]:
shows_links_df = shows_links_df[shows_links_df['Network']=='Netflix']

shows_links_df = shows_links_df.loc[:, ["Show", "Critic Score", "Audience Score"]]

shows_links_df['title_lower'] = shows_links_df['Show'].str.lower()

for column in ["Critic Score", "Audience Score"]:
    shows_links_df[column] = shows_links_df[column].str.replace('%', '').str.replace('--', '-1')

if column in ['Show', 'title_lower']:
    shows_links_df[column] = shows_links_df[column].astype(str)
else: 
    shows_links_df[column] = shows_links_df[column].astype(int)

shows_links_df = shows_links_df.rename(columns={'Show': 'title', "Critic Score": 'critic_score', "Audience Score": 'audience_score'})

shows_links_df = shows_links_df.loc[:, ["title", "title_lower", "critic_score", "audience_score"]]

shows_links_df.reset_index(drop=True, inplace=True)

shows_links_df.head(5)

,title,title_lower,critic_score,audience_score
0,The Umbrella Academy,the umbrella academy,85,78
1,Man vs. Bee,man vs. bee,62,62
2,Stranger Things,stranger things,91,91
3,Peaky Blinders,peaky blinders,94,93
4,You Don't Know Me,you don't know me,-1,-1


<h3> Przygotowanie Zbioru Danych z Recenzjami Krytyków 

In [10]:
critics_reviews_df = critics_reviews_df.merge(shows_links_df[['title','title_lower']],left_on='Show',right_on='title',how='left')

critics_reviews_df = critics_reviews_df.dropna()

critics_reviews_df = critics_reviews_df.loc[:, ['Show', 'title_lower', 'Review', 'Sentiment']]

critics_reviews_df = critics_reviews_df.rename(columns={'Show': 'title', 'Review': 'review', 'Sentiment': 'sentiment'})

for column in ['title', 'title_lower', 'review']:
    critics_reviews_df[column] = critics_reviews_df[column].astype(str)

for column in integer_fields:
    critics_reviews_df['sentiment'] = critics_reviews_df["sentiment"].astype(int)

critics_reviews_df.reset_index(drop=True, inplace=True)

critics_reviews_df.head(2)

,title,title_lower,review,sentiment
0,Avatar: The Last Airbender,avatar: the last airbender,A stalwart example of allegory's power to info...,1
1,Avatar: The Last Airbender,avatar: the last airbender,"Great characters, detailed fantasy worldbuildi...",1


<h3> Przygotowanie Zbioru Danych z Recenzjami Widzów 

In [11]:
users_reviews_df = users_reviews_df.merge(shows_links_df[['title','title_lower']],left_on='Show',right_on='title',how='left')

users_reviews_df = users_reviews_df.dropna()

users_reviews_df = users_reviews_df.loc[:, ['Show', 'title_lower', 'Review', 'Rating']]

users_reviews_df = users_reviews_df.rename(columns={'Show': 'title', 'Review': 'review', 'Rating': 'rating'})

for column in ['title', 'title_lower', 'review']:
    users_reviews_df[column] = users_reviews_df[column].astype(str)

users_reviews_df['rating'] = users_reviews_df["rating"].astype(float)

users_reviews_df.reset_index(drop=True, inplace=True)

users_reviews_df.head(2)

,title,title_lower,review,rating
0,Avatar: The Last Airbender,avatar: the last airbender,"This is a show for children, yet it is far mor...",5.0
1,Avatar: The Last Airbender,avatar: the last airbender,This is overall a pretty amazing show. I did f...,4.5


<h3> Przygotowanie Zbioru Danych z Opiniami o Aplikacji 

In [13]:
app_reviews_df = pd.read_csv(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_App_Reviews_Data.csv")

for column in ['review_id', 'pseudo_author_id']: 
    app_reviews_df = app_reviews_df.drop_duplicates(column)

app_reviews_df = app_reviews_df.dropna()

app_reviews_df = app_reviews_df.loc[:, ["review_id", "pseudo_author_id", "review_text", "review_rating", "review_likes", "review_timestamp"]]

app_reviews_df = app_reviews_df.rename(columns={"pseudo_author_id": 'reviewer_id', "review_text": 'review', "review_timestamp": 'review_date'})

for column in ["review_id", "reviewer_id", "review", "review_date"]:
    app_reviews_df[column] = app_reviews_df[column].astype(str)

for column in ["review_rating", "review_likes"]:
    app_reviews_df[column] = app_reviews_df[column].astype(int)

app_reviews_df['review_date'] = app_reviews_df['review_date'].str[:10]

app_reviews_df['review_date'] = pd.to_datetime(app_reviews_df['review_date'])

app_reviews_df.head(2)

,review_id,reviewer_id,review,review_rating,review_likes,review_date
0,7e73f80e-a8fd-4ff3-b09b-502f0ad058ff,152618553977019693742,Works great on my Evo! Glad android phones are...,5,1,2011-05-12
1,dab55eca-c2a0-45a8-93e3-9860c1c548da,234382942865437071667,Works great on HTC incredible. Can't wait to t...,5,1,2011-05-12


<h3> Eksport plików

In [ ]:
content_df.to_xlsx(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_Productions_Data_processed.xlsx")
app_reviews_df.to_xlsx(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_App_Reviews_Data_processed.xlsx")
users_reviews_df.to_xlsx(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_Users_Reviews_processed.xlsx")
critics_reviews_df.to_xlsx(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_Critics_Reviews_processed.xlsx")
shows_links_df.to_xlsx(r"/Users/kasiabonter/Desktop/Netflix Datasets/Netflix_TV_Shows_Links_processed.xlsx")